In [ ]:
import pandas as pd

### Criar DataFrames com base nos datasets

In [ ]:
# Carregar Dataframe de clientes (customers)
df_customers = pd.read_csv('./datasets/churn_customers.csv')

In [ ]:
df_contracts = pd.read_csv('./datasets/churn_contracts.csv')

In [ ]:
df_services = pd.read_csv('./datasets/churn_services.csv')

In [ ]:
df_customers.head(5)

In [ ]:
df_customers.tail(5)

In [ ]:
df_customers.describe()

In [ ]:
df_customers.info()

### Transformação de dados

In [ ]:
# Transformar a coluna "TotalCharges" de string para float - errors 'coerce' -> se não consegue converter para numérico, converte para None (null)
df_contracts['TotalCharges'] = pd.to_numeric(df_contracts['TotalCharges'], errors='coerce')

In [ ]:
df_contracts.info()

### Renomear colunas

In [ ]:
df_customers.info()

In [ ]:
# Renomear coluna SeniorCitizen. O rename retorna uma cópia do dataframe, com o rename aplicado. O arg "inplace" faz com que o dataframe atual seja renomeado
df_customers.rename(columns={'SeniorCitizen': 'Above65yo'}, inplace=True)

In [ ]:
df_customers.info()

In [ ]:
# Renomear usando Listas
df_customers.columns = ['id', 'genero', 'idoso', 'casado', 'dependentes']

In [ ]:
df_customers.info()

### Unificar Dataframes customers, services e contracts

In [ ]:
# Contar quantidade de registros por dataframe
len(df_customers)

In [ ]:
len(df_contracts)

In [ ]:
len(df_services)

In [ ]:
df_contracts.info()

In [ ]:
df_contracts.rename(columns={'customerID': 'id'}, inplace=True)

In [ ]:
df_contracts.info()

In [ ]:
df_services.rename(columns={'customerID': 'id'}, inplace=True)

In [ ]:
df_services.info()

In [ ]:
df_temp = df_contracts.merge(df_services, on=['id'])

In [ ]:
df_temp.head(5)

In [ ]:
df_temp.info()

In [ ]:
df_customers.rename(columns={'id': 'customerId'}, inplace=True)

In [ ]:
# Unificar df_temp com customers utilizando colunas de junção de nomes distintos (duplica a coluna comum, devido ao nome distinto)
df_churn_temp = df_temp.merge(df_customers, left_on=['id'], right_on=['customerId'], )

In [ ]:
df_churn_temp.info()

In [ ]:
df_churn_temp.head(5)

In [ ]:
# Unificar os três dataframes ao mesmo tempo contendo colunas de junção de nomes distintos
df_churn = df_customers.merge(df_services, left_on=['customerId'], right_on=['id']).merge(df_contracts, on=['id'])

In [ ]:
df_churn.info()

In [ ]:
# Remover coluna duplicada
df_churn.drop(columns=['id'], inplace=True)

In [ ]:
df_churn.rename(columns={'customerId': 'id'}, inplace=True)

In [ ]:
df_churn.info()

### Detecção de valores ausentes

In [ ]:
# Detectar valores ausentes em todas as colunas de um dataframe
df_churn.isna().sum()

In [ ]:
# Detectar valores ausentes em uma coluna
df_churn['TotalCharges'].isna().sum()

In [ ]:
# Quantas linhas têm pelo menos uma coluna com valor ausente
df_churn[df_churn.isna().any(axis=1)]

In [ ]:
# Quais colunas tem ao menos um valor ausente
df_churn.isna().any(axis=0)

### Remover valores ausentes

In [ ]:
# Remover de forma direta e específica a coluna que possuem valores ausentes
df_churn.drop(columns=['TotalCharges'], axis=1)

In [ ]:
# Remover todas as colunas que possuem ao menos um valor ausente
df_churn.dropna(axis=1)

In [ ]:
# Remover linhas com dados nulos
df_churn.dropna(axis=0)

In [ ]:
# Remover linhas caso todos seus valores sejam ausentes
df_churn.dropna(axis=0, how='all')

### Imputação de valores ausentes

In [ ]:
# Preencher todos os valores ausentes pelo número 0
df_churn.fillna(0)

In [ ]:
# Preencher valores padrão conforme a coluna, considerando valores ausentes
df_churn.fillna(value={'TotalCharges': 0, 'genero': 'Não declarado' })

In [ ]:
# Preencher todos os valores ausentes de uma coluna com a média
media_TotalCharges = df_churn['TotalCharges'].mean()

In [ ]:
media_TotalCharges

In [ ]:
df_churn.fillna(value={'TotalCharges': media_TotalCharges})

### Análise Univariada

- A faixa etária do cliente tem uma forte associação com o churn
- Um cliente com menos de 6 meses de contrato é mais propenso ao churn
- Um cliente com contrato mensal é mais propenso ao churn

In [ ]:
# Identificar valores possíveis/únicos em uma variável do dataframe
df_churn['Churn'].unique()

In [ ]:
# Contar clientes usando a variável Churn como referência, agrupando pelos valores possíveis
df_churn['Churn'].value_counts()

In [ ]:
# Como é a distribuição percentual de clientes que abandonaram ou continuam ativos
df_churn['Churn'].value_counts(normalize=True)

In [ ]:
# Plot distribuição do churn (quantidade)
ax = df_churn['Churn'].value_counts().plot.bar()
ax.bar_label(ax.containers[0])

In [ ]:
# Plot distribuição do churn (percentual)
ax = df_churn['Churn'].value_counts(normalize=True).plot.bar()
ax.bar_label(ax.containers[0])

In [ ]:
# Quais são os tipos de contrato
df_churn["Contract"].value_counts()

In [ ]:
# Plot distribuição dos contratos (quantidade)
ax = df_churn['Contract'].value_counts().plot.bar()
ax.bar_label(ax.containers[0])

In [ ]:
# Plot distribuição dos contratos (percentual)
ax = df_churn['Contract'].value_counts(normalize=True).plot.bar()
ax.bar_label(ax.containers[0])

In [ ]:
# Tempo de contrato (meses)
df_churn["tenure"].plot.hist()

- A variável tenure (tempo de contrato) não se apresenta visualmente como uma distribuição normal
- A grande parte dos valores se encontram nos extremos, ou seja, contratos de curta duração e contratos de longa duração, acima dos 65 meses

In [ ]:
# Histograma do MonthlyChargest (custom mensal do contrato)
df_churn['MonthlyCharges'].plot.hist()

In [ ]:
# Medidas de posição - Média do tempo de contrato
df_churn['tenure'].mean()

In [ ]:
# Medidas de posição - Mediana do tempo de contrato
df_churn['tenure'].median()

In [ ]:
# Medidas de posição - Moda do tempo de contrato
df_churn['tenure'].mode()[0]

In [ ]:
# Medidas de dispersão - Desvio padrão
df_churn['tenure'].std()

In [ ]:
# Medidas de dispersão - Coeficiente de variação
df_churn['tenure'].std() / df_churn['tenure'].mean() * 100

In [ ]:
# Quantos clientes possuem um mês de contrato - Filtro
len(df_churn[(df_churn['tenure'] == 1)])

In [ ]:
# Quantos os clientes de um mês de contrato representam percentualmente?
len(df_churn[(df_churn['tenure'] == 1)]) / len(df_churn) * 100

In [ ]:
# Quantos clientes têm entre um e seis meses de contrato?
df_churn[(df_churn['tenure'] >= 1) & (df_churn['tenure'] <= 6)]['tenure'].value_counts()

In [ ]:
# Aprensentar a quantidade de clientes por tempo de contrato
df_churn.groupby('tenure')['tenure'].count().sort_values(ascending=False)

In [ ]:
# Apresentar a quantidade de clientes - Agrupamento com sumarização e plot
df_churn.groupby('tenure')['tenure'].count().sort_values().plot.barh(figsize=(20,20))

### Análise Bivariada

In [ ]:
# Hipótese: clientes com contrato mensal estão mais propensos ao churn - construir uma tabela de contingência
pd.crosstab(df_churn['Churn'], df_churn['Contract'], margins=True, margins_name='Total')

In [ ]:
# Construir a tabela de contingência entre tipo de contrato e churn - proporção
pd.crosstab(df_churn['Churn'], df_churn['Contract'], normalize=True, margins=True, margins_name='Total')

In [ ]:
# Construir a tabela de contingência entre tipo de contrato e churn - proporção com totalização por linha
pd.crosstab(df_churn['Churn'], df_churn['Contract'], normalize='index', margins=True, margins_name='Total')

- Considerando o público total, observamos que, dos clientes que deixaram o serviço, 88.5% possuíam contrato mensal. Desta forma, podemos afirmar que há uma associação entre tipo de contrato mensal e o abandono do serviço.

##### Avaliar correlação entre duas variáveis qualitativas (ou categóricas)
- Executar um teste de hipótese chamado chi-square de Pearson (ou qui-quadrado): Num teste de hipótese, duas hipóteses são formuladas
    - H0 (Hipótese nula): as duas variáveis são independentes
    - H1 (Hipotese complementar): as duas variáveis não são independentes

- O teste serve para confirmar ou recusar a hipótese nula (H0)
- Quando a probabilidade de observarmos H0 é inferior ao p-value (por convenção, 0.05), recusamos H0 e seguimos com a H1


In [ ]:
# Gerar um dataframe da crosstab (sem totais)
df_crosstab_churn_contract = pd.crosstab(df_churn['Churn'], df_churn['Contract'])

In [ ]:
# Calcular os scores e p-values
from scipy.stats import chi2_contingency

In [ ]:
chi_scores_churn_contract = chi2_contingency(df_crosstab_churn_contract)

In [ ]:
chi_scores_churn_contract

In [ ]:
scores_churn_contract = pd.Series(chi_scores_churn_contract[0])

In [ ]:
pvalues_churn_contract = pd.Series(chi_scores_churn_contract[1])

In [ ]:
# Apresentar números com decimais sem a notação científica
pd.set_option('display.float_format', lambda x: '%.15f' % x)

In [ ]:
# Criar um dataframe com os resultados
df_chi_scores_churn_contract = pd.DataFrame({'Chi2': scores_churn_contract, 'p-value': pvalues_churn_contract})

In [ ]:
df_chi_scores_churn_contract

- Conforme o teste estatístico chi-square, o p-value <= 0.05, desta forma rejeitamos a hipótese nula, ou seja, as variáveis não são independentes. Pelo chi2 alto, podemos afirmar que há uma forte correlação

In [ ]:
# Analisar a correlação entre uma variável qualitativa e outra quantitativa: utilizar o mesmo teste chi-square, transformando a quantitativa em qualitativa
# Hipótese: cliente com menos de 6 meses de contrato é mais propenso ao churn
# Criar uma variável Yes/No para contratos abaixo ou acima de 6 meses. Criar uma nova coluna com base na condição de outra coluna - LessThan6MonthsContract
import numpy as np

In [ ]:
df_churn['LessThan6MonthsContract'] = np.where(df_churn['tenure'] < 6, 'Yes', 'No')

In [ ]:
df_churn.head(5)

In [ ]:
tenure_churn_crosstab = pd.crosstab(df_churn['Churn'], df_churn['LessThan6MonthsContract'])

In [ ]:
chi_scores_tenure_churn = chi2_contingency(tenure_churn_crosstab)

In [ ]:
scores_tenure_churn = pd.Series(chi_scores_tenure_churn[0])
pvalues_tenure_churn = pd.Series(chi_scores_tenure_churn[1])
df_tenure_churn = pd.DataFrame({'chi2': scores_tenure_churn, 'p-value': pvalues_tenure_churn})

In [ ]:
df_tenure_churn

- Com o p-value < 0.05, rejeitamos a H0, ou seja, as variáveis não são independentes. Com base no chi2 resultante dos dois testes, a correlação entre churn e tempo de contrato inferior a 6 meses é alta, mas é menor que a correlação entre churn e tipo de contrato

In [ ]:
# Correlação entre duas variáveis quantitativas
# Hipótese: Quanto maior o tempo de contrato (tenure), maior o total pago pelo cliente (TotalCharges)
# Coeficiente de correlação de Pearson
df_churn['tenure'].corr(df_churn['TotalCharges'])

In [ ]:
# Coeficiente de correlação de Spearman (captura correlações que podem não ser lineares)
df_churn['tenure'].corr(df_churn['TotalCharges'], method='spearman')

- Há uma forte correlação entre tempo de contrato (tenure) e total pago pelo cliente (TotalCharges), de acordo com os coeficientes de correlação de Pearson e Spearman

In [ ]:
# Apresentar plot scatter entre tenure e TotalCharges (gráfico de dispersão)
df_churn.plot.scatter(x='tenure', y='TotalCharges')

- Dado os serviços, o valor pago muda. Porém, observa-se uma tendência de crescimento de acordo com a Tenure (linhas inferiores e superiores bem definidas)
- Uma análise multivariada pode ser interessante neste caso, adicionando os serviços contratados na análise

- Desafio 1: validar a primeira hipótese - Faixa etária tem forte associação com o churn
- Desafio 2: validar com o teste de hipóteses se o contrato mensal está mais propenso ao churn

In [ ]:
# Desafio 1: validar a primeira hipótese - Faixa etária tem forte associação com o churn
idoso_churn_crosstable = pd.crosstab(df_churn['idoso'], df_churn['Churn'])

In [ ]:
chi_scores_idoso_churn = chi2_contingency(idoso_churn_crosstable)

In [ ]:
scores_idoso_churn = pd.Series(chi_scores_idoso_churn[0])
pvalue_idoso_churn = pd.Series(chi_scores_idoso_churn[1])
df_idoso_churn = pd.DataFrame({'chi2': scores_idoso_churn, 'p-value': pvalue_idoso_churn})

In [ ]:
df_idoso_churn

- Rejeitada a H0, temos que existe correlação entre entre churn e faixa etária (senioridade), sendo esta mais fraca do que tempo de contrato (tenure) e total pago (TotalCharges)

In [ ]:
# Desafio 2: Validar com teste de hipóteses se o contrato mensal está mais propenso ao churn
# Categorizar se é ou não contrato mensal
df_churn['MonthlyContract'] = np.where(df_churn['Contract'] == 'Month-to-month', 'Yes', 'No')

In [ ]:
df_churn.head(5)

In [ ]:
monthlycontract_churn_crosstable = pd.crosstab(df_churn['MonthlyContract'], df_churn['Churn'])

In [ ]:
chi_scores_monthlycontract_churn = chi2_contingency(monthlycontract_churn_crosstable)

In [ ]:
scores_monthlycontract_churn = pd.Series(chi_scores_monthlycontract_churn[0])
pvalues_monthlycontract_churn = pd.Series(chi_scores_monthlycontract_churn[1])
df_monthlycontract_churn = pd.DataFrame({'chi2': scores_monthlycontract_churn, 'p-value': pvalues_monthlycontract_churn})

In [ ]:
df_monthlycontract_churn

- Portanto, reafirmamos a conclusão que já possuimos: os dados de Churn e de MonthlyContracts (contratos mensais) não são independentes, tendo uma relação mais forte do que tempo de contrato e senioridade

### Detecçaõ de Outliers

In [ ]:
df_churn.info()

In [ ]:
# Remover linhas com valores ausentes
df_churn.dropna(axis=0, inplace=True)

In [ ]:
# Box plot geral
df_churn['TotalCharges'].plot.box()

In [ ]:
# Box plot agrupado por Contract
df_churn.plot.box(column='TotalCharges', by='Contract')

In [ ]:
# Criar dataframe somente para clientes com contrato do tipo Month-to-month
df_churn_month = df_churn[df_churn['Contract'] == 'Month-to-month']

In [ ]:
df_churn_month

In [ ]:
# Histograma - verificar se a distribuição tende à normal
df_churn_month['TotalCharges'].plot.hist()

In [ ]:
# Método de Tukey - IQR (distribuição não normal)
# IQR: Interquartile range = 3º quartil - 1º quartil
q1_TotalCharges_month = df_churn_month['TotalCharges'].quantile(0.25)
q3_TotalCharges_month = df_churn_month['TotalCharges'].quantile(0.75)
iqr_TotalCharges_month = q3_TotalCharges_month - q1_TotalCharges_month
iqr_TotalCharges_month


In [ ]:
# Limites inferior e superior. Caso o inferior seja negativo, mas isso não se aplique, assume-se zero.
limInf_TotalCharges_month = q1_TotalCharges_month - (iqr_TotalCharges_month * 1.5)
limSup_TotalCharges_month = q3_TotalCharges_month + (iqr_TotalCharges_month * 1.5)
limInf_TotalCharges_month, limSup_TotalCharges_month

In [ ]:
# Filtro que retorna os outliers
df_churn_month[(df_churn_month['TotalCharges'] < limInf_TotalCharges_month) | (df_churn_month['TotalCharges'] > limSup_TotalCharges_month)]

In [ ]:
# Método ZScore: Utilizado para distribuição normal
# Indica quantos desvios-padrões um ponto específico está distante da média
# zscore = (x - média)/desvio padrão
from scipy.stats import zscore

In [ ]:
z = np.abs(zscore(df_churn_month['TotalCharges']))
z

In [ ]:
# Valor comum: valores acima ou abaixo de 3 desvios padrões da média é um outlier
df_churn_month[z > 3.0]

### Automatizando EDA com Sweetviz

In [ ]:
# Sweetviz - Abrir no Jupyter Notebook
import sweetviz as sv
sv_churn_report = sv.analyze(df_churn, target_feat='Churn')

In [ ]:
sv_churn_report.show_notebook()

In [ ]:
sv_churn_report.show_html()